In [2]:
import pandas as pd
import requests
import random
import json
import time
import glob
import os

In [1]:
tool2tag = {
    'Azure Machine Learning': {'azure-machine-learning-service', 'azure-machine-learning-studio', 'azure-machine-learning-workbench'},
    'Vertex AI': {'google-cloud-vertex-ai', 'vertex-ai-pipeline'},
    'Amazon SageMaker': {'amazon-sagemaker', 'amazon-sagemaker-experiments'},
    'MLFlow': {'mlflow'},
    'DVC': {'dvc'},
    'Weights & Biases': {'wandb'},
    'ClearML': {'clearml'},
    'Comet': {'comet-ml'},
    'Neptune': {'neptune'},
    'Pachyderm': {'pachyderm'}
}

In [3]:
path_dataset = '../Dataset'

path_so = os.path.join(path_dataset, 'Stack Overflow')
path_to = os.path.join(path_dataset, 'Tool-specific Others')
    
path_so_raw = os.path.join(path_so, 'Raw')
path_to_raw = os.path.join(path_to, 'Raw')
path_so_filtered = os.path.join(path_so, 'Filtered')
path_to_filtered = os.path.join(path_to, 'Filtered')
path_so_sampled = os.path.join(path_so, 'Sampled')
path_to_sampled = os.path.join(path_to, 'Sampled')

if not os.path.exists(path_dataset):
    os.makedirs(path_dataset)

if not os.path.isdir(path_so):
    os.mkdir(path_so)
    
if not os.path.isdir(path_to):
    os.mkdir(path_to)
    
if not os.path.isdir(path_so_raw):
    os.mkdir(path_so_raw)
    
if not os.path.isdir(path_to_raw):
    os.mkdir(path_to_raw)
    
if not os.path.isdir(path_so_filtered):
    os.mkdir(path_so_filtered)
    
if not os.path.isdir(path_to_filtered):
    os.mkdir(path_to_filtered)
    
if not os.path.isdir(path_so_sampled):
    os.mkdir(path_so_sampled)
    
if not os.path.isdir(path_to_sampled):
    os.mkdir(path_to_sampled)

In [8]:
page = -1
base_url = 'https://my.guild.ai/'
page_suffix = 'c/troubleshooting/6.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_raw, 'Guild AI.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [7]:
page = -1
base_url = 'https://community.wandb.ai/'
page_suffix = 'c/w-b-support/36.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_raw, 'Weights & Biases.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [5]:
page = -1
base_url = 'https://community.sigopt.com/'
page_suffix = 'c/general-discussion/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_raw, 'SigOpt.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [5]:
page = -1
base_url = 'https://discuss.dvc.org/'
page_suffix = 'c/questions/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_raw, 'DVC.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [4]:
# filter out tool-specific posts with negative upvote count
for file_name in glob.glob(os.path.join(path_to_raw, '*.json')):
    repos = pd.read_json(file_name)
    if 'Question_upvote_count' in repos.columns:
        repos = repos[repos['Question_upvote_count'] > -1]
    tool_name = os.path.split(file_name)[1].split('.')[0]
    repos.to_json(os.path.join(path_to_filtered, f'{tool_name}.json'), orient='records', indent=4)

In [11]:
df = pd.read_json(os.path.join(path_so_raw, 'bq-results-20221220-235957-1671580809671.json'), lines=True)
df.head()

,Question_id,Question_title,Question_body,Question_answer_count,Question_comment_count,Question_creation_date,Question_favorite_count,Question_last_edit_date,Question_score,Question_tags,...,Owner_reputation,Owner_up_votes,Owner_down_votes,Owner_views,Answer_body,Answer_comment_count,Answer_creation_date,Answer_score,Owner_location,Answer_last_edit_date
0,3037664,"Does any Version Control System like SVN, Git,...","<p>In our project files, if there are binary f...",10,3,2010-06-14 13:42:06.327 UTC,1.0,2010-06-14 22:18:01.693 UTC,10,svn|git|mercurial|dvcs,...,141372,1451,39,12992,"<p>I do know one that does this, but you're no...",0.0,2010-07-13 22:17:41.03 UTC,4.0,NaN,NaN
1,29647782,Azureml Web Service - How to create a Rest Ser...,<p>I've looked all over the google and stackov...,1,0,2015-04-15 10:26:48.083 UTC,NaN,2015-04-16 19:48:33.29 UTC,1,json|web-services|azure|webforms|azure-machine...,...,23,0,0,9,NaN,NaN,NaN,NaN,NaN,NaN
2,60784209,Sagemaker Neo Compilation witn Dynamic Input D...,<p>I'm trying to compile a <code>PyTorch</code...,0,0,2020-03-21 03:31:41.6 UTC,NaN,NaN,1,amazon-web-services|pytorch|amazon-sagemaker,...,2306,1283,3,182,NaN,NaN,NaN,NaN,"Montreal, QC, Canada",NaN
3,70494276,Not able to read HDF5 file present in S3 in sa...,<p>My directory structure looks like this: <co...,1,0,2021-12-27 10:26:41.04 UTC,NaN,NaN,0,amazon-s3|hdf5|amazon-sagemaker,...,1,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN
4,51940106,What is the best practice to develop CD/CI whe...,"<p>In our backend development process, we have...",1,0,2018-08-21 00:33:31.857 UTC,NaN,2018-08-21 03:35:00.183 UTC,0,azure-machine-learning-studio|ml-studio,...,37,2,0,8,NaN,NaN,NaN,NaN,Australia,NaN


In [13]:
# create tag collection
tags = []
for key, value in tool2tag.items():
    tags.extend(value)

In [14]:
# split tags 
df['Question_valid_tags'] = [[] for _ in range(len(df))]
for index, row in df.iterrows():
    row_tags = set(str(row['Question_tags']).strip().split('|'))
    df.at[index, 'Question_valid_tags'] = list(row_tags.intersection(tags))

In [15]:
# count post number with different tags
arity = 0
while True:
    post_number = df[df['Question_valid_tags'].map(len) > arity].shape[0]
    if post_number < 1:
        break
    arity = arity + 1
    print(f'Posts with at least {arity} tags has {post_number} in total.')

Posts with at least 1 tags has 4981 in total.
Posts with at least 2 tags has 213 in total.
Posts with at least 3 tags has 16 in total.


In [20]:
# filter out Stack Overflow posts with unrelated tags
df_valid = df[df['Question_valid_tags'].map(len) > 0]
df_valid['Question_link'] = df_valid['Question_id'].apply(lambda x: f'https://stackoverflow.com/questions/{x}')
df_valid.to_json(os.path.join(path_so_filtered, 'valid_posts.json'),
                 indent=4, orient='records')

C:\Users\zhimi\AppData\Local\Temp\ipykernel_18264\2928942611.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid['Question_link'] = df_valid['Question_id'].apply(lambda x: f'https://stackoverflow.com/questions/{x}')


In [22]:
# filter out Stack Overflow posts with negative upvote count
df_qualified = df_valid[df_valid['Question_score'] > -1]
df_qualified.to_json(os.path.join(
    path_so_filtered, 'valid_posts_with_at_least_zero_score.json'), indent=4, orient='records')

In [23]:
# filter out Stack Overflow posts without accepted answers
df_qualified_accepted = df_qualified.dropna(subset=['Answer_body'])
df_qualified_accepted.to_json(os.path.join(
    path_so_filtered, 'valid_posts_with_at_least_zero_score_and_answer_accepted.json'), indent=4, orient='records')

In [24]:
# check how the posts with more than one tags look like
df_one_plus_tags = df[df['Question_valid_tags'].map(len) > 1]
df_one_plus_tags['Question_valid_tags'].map(str).unique().tolist()

["['azure-machine-learning-studio', 'azure-machine-learning-service']",
 "['azure-machine-learning-workbench', 'azure-machine-learning-studio', 'azure-machine-learning-service']",
 "['azure-machine-learning-workbench', 'azure-machine-learning-studio']",
 "['amazon-sagemaker', 'mlflow']",
 "['mlflow', 'dvc']",
 "['mlflow', 'azure-machine-learning-service']",
 "['dvc', 'pachyderm']",
 "['azure-machine-learning-workbench', 'azure-machine-learning-service']",
 "['mlflow', 'azure-machine-learning-studio']"]

In [25]:
# create a map from tag to tool
tag2tool = {}
for key, value in tool2tag.items():
    for elem in value:
        tag2tool.setdefault(elem, key)

In [26]:
# extract Stack Overflow post collection with multiple tags based on the tool map
for index, row in df_qualified.iterrows():
    tags = set()
    for tag in row['Question_valid_tags']:
        tags.add(tag2tool[tag])
    df_qualified.at[index, 'Question_valid_tags'] = sorted(list(tags))

df_multiply_tagged = df_qualified[df_qualified['Question_valid_tags'].map(len) > 1]
df_multiply_tagged.to_json(os.path.join(path_so_filtered, 'valid_posts_with_at_least_zero_score_and_multiple_tags.json'), indent=4, orient='records')

In [27]:
# create Stack Overflow post collection with exclusive tags
multiply_tagged_posts_split = []
df_qualified.assign(Question_exclusive_tag='')

for index, row in df_qualified.iterrows():
    tags = row['Question_valid_tags']
    df_qualified.at[index, 'Question_exclusive_tag'] = tags[0]
    if len(tags) > 1:
        for tag in tags[1:]:
            series = row.copy()
            series['Question_exclusive_tag'] = tag
            multiply_tagged_posts_split.append(series)
            
df_multiply_tagged_posts_split = pd.DataFrame(multiply_tagged_posts_split)
df_qualified_exclusive_tagged = pd.concat([df_qualified, df_multiply_tagged_posts_split], ignore_index=True)
del df_qualified_exclusive_tagged['Question_valid_tags']
df_qualified_exclusive_tagged.to_json(os.path.join(path_so_filtered, 'valid_posts_with_at_least_zero_score_and_one_exclusive_tag.json'), indent=4, orient='records')

C:\Users\zhimi\AppData\Local\Temp\ipykernel_18264\2280223683.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qualified.at[index, 'Question_exclusive_tag'] = tags[0]


In [28]:
df_qualified_exclusive_tagged = pd.read_json(os.path.join(path_so_filtered, 'valid_posts_with_at_least_zero_score_and_one_exclusive_tag.json'))

In [29]:
df_summary = df_qualified_exclusive_tagged.groupby('Question_exclusive_tag').count()['Question_title']
df_summary = pd.DataFrame(df_summary).reset_index()
df_summary.columns = ['Tool', '#Stack Overflow Posts']
df_summary['#Tool-specific Posts'] = 0

for index, row in df_summary.iterrows():
    try:
        df_tool = pd.read_json(os.path.join(path_to_filtered, f'{row["Tool"]}.json'))
        df_summary.loc[index, '#Tool-specific Posts'] = df_tool.shape[0]
    except:
        pass

df_summary['%Stack Overflow Posts'] = 0
df_summary['%Tool-specific Posts'] = 0

df_summary['%Stack Overflow Posts'] = df_summary['#Stack Overflow Posts'] / (df_summary['#Stack Overflow Posts'] + df_summary['#Tool-specific Posts']) * 100
df_summary['%Tool-specific Posts'] = 100 - df_summary['%Stack Overflow Posts']

df_summary['%Stack Overflow Posts'] = round(df_summary['%Stack Overflow Posts'], 2)
df_summary['%Tool-specific Posts'] = round(df_summary['%Tool-specific Posts'], 2)

# After having the population for each tool and discussion channel, we then find out the minimum number of necessary samples with the [calculator](https://www.calculator.net/sample-size-calculator.html).
df_summary.to_csv(os.path.join(path_dataset, 'summary.csv'), index=False)
df_summary

,Tool,#Stack Overflow Posts,#Tool-specific Posts,%Stack Overflow Posts,%Tool-specific Posts
0,Amazon SageMaker,2233,528,80.88,19.12
1,Azure Machine Learning,1530,1435,51.60,48.40
2,ClearML,40,0,100.00,0.00
3,Comet,10,0,100.00,0.00
4,DVC,91,315,22.41,77.59
5,MLFlow,551,280,66.31,33.69
6,Neptune,8,0,100.00,0.00
7,Pachyderm,7,0,100.00,0.00
8,Vertex AI,341,297,53.45,46.55
9,Weights & Biases,77,583,11.67,88.33


In [30]:
df_summary = pd.read_csv(os.path.join(path_dataset, 'summary.csv'))
df_summary

,Tool,#Stack Overflow Posts,#Tool-specific Posts,%Stack Overflow Posts,%Tool-specific Posts,#Stack Overflow Samples,#Tool-specific Samples
0,Amazon SageMaker,2233,528,80.88,19.12,328,223
1,Azure Machine Learning,1530,1435,51.60,48.40,308,304
2,ClearML,40,0,100.00,0.00,37,0
3,Comet,10,0,100.00,0.00,10,0
4,DVC,91,315,22.41,77.59,74,174
5,MLFlow,551,280,66.31,33.69,227,163
6,Neptune,8,0,100.00,0.00,8,0
7,Pachyderm,7,0,100.00,0.00,7,0
8,Vertex AI,341,297,53.45,46.55,181,168
9,Weights & Biases,77,583,11.67,88.33,65,232


In [7]:
# sample the tool-specific posts
for index, row in df_summary.iterrows():
    if not row['#Tool-specific Posts']:
        continue
    df_to = pd.read_json(os.path.join(path_to_filtered, f'{row["Tool"]}.json'))
    df_to_sample = df.sample(n=row['#Tool-specific Samples'], random_state=0)
    df_to_sample.to_json(os.path.join(path_to_sampled, f'{row["Tool"]}.json'), indent=4, orient='records')

In [19]:
# sample the Stack Overflow posts
df_so = pd.read_json(os.path.join(path_so_filtered, 'valid_posts_with_at_least_zero_score_and_one_exclusive_tag.json'))
for index, row in df_summary.iterrows():
    if not row['#Stack Overflow Samples']:
        continue
    df_so_sample = df_so[df_so['Question_exclusive_tag'] == row['Tool']].sample(n=row['#Stack Overflow Samples'], random_state=0)
    df_so_sample.to_json(os.path.join(path_so_sampled, f'{row["Tool"]}.json'), indent=4, orient='records')